In [29]:
import pandas as pd

In [30]:
df = pd.read_csv(r"/home/ali/Desktop/GA-new-Version/dataset/course_dataset.csv")

In [19]:
df

,Course ID,Section,Teacher ID,Credit Hours,Semester
0,CE413,A,Mr. Salman Saeed,3,8
1,CE413,B,Mr. Salman Saeed,3,8
2,CS439,A,Dr. Farhan Khan,3,8
3,CS440,B,Dr. Farhan Khan,3,8
4,CS441,C,Dr. Farhan Khan,3,8
...,...,...,...,...,...
98,CS373,A,Dr. Fahad Bin Muslim,3,6
99,CS374,B,Dr. Fahad Bin Muslim,3,6
100,CS375,C,Dr. Fahad Bin Muslim,3,6
101,CS376,D,Dr. Fahad Bin Muslim,3,6


In [ ]:
# Python3 program to create target string, starting from 
# random string using Genetic Algorithm 
  
import random 
from collections import Counter

# Number of individuals in each generation 
POPULATION_SIZE = 100
  
  
class Individual(object): 
    ''' 
    Class representing individual in population 
    '''
    def __init__(self, chromosome): 
        self.chromosome = chromosome  
        self.fitness = self.cal_fitness() 

    @classmethod
    def create_gnome(self): 
        ''' 
        create chromosome or string of genes 
        '''
        
        week = []
        for i in range(50):
            lecture_hall = []
            for j in range(0,8):
                    lecture_hall.append(random.randint(0, 102))
            week.append(lecture_hall)
            
        return week
  
    def mate(self, par2): 
        ''' 
        Perform mating and produce new offspring 
        '''
  
        # chromosome for offspring 
        child_chromosome = [] 
        for gp1, gp2 in zip(self.chromosome, par2.chromosome):     
  
            # random probability   
            prob = random.random() 
  
            # if prob is less than 0.45, insert gene 
            # from parent 1  
            if prob < 0.45: 
                child_chromosome.append(gp1) 
  
            # if prob is between 0.45 and 0.90, insert 
            # gene from parent 2 
            elif prob < 0.90: 
                child_chromosome.append(gp2) 
  
            # otherwise insert random gene(mutate),  
            # for maintaining diversity 
            else: 
                lecture_hall = []
                for j in range(0,8):
                    lecture_hall.append(random.randint(0, 102))
                child_chromosome.append(lecture_hall) 
  
        # create new Individual(offspring) using  
        # generated chromosome for offspring 
        return Individual(child_chromosome) 
  
    def cal_fitness(self):
        
        # check for unique classes
        fitness  = 0 
        temp_list = []
        ctr = 0
        for chromosom in self.chromosome:
            temp_list.append(chromosom)
            ctr += 1
            if ctr%10 == 0 and ctr!=0:
                flatList = [element for innerList in temp_list for element in innerList]
                counts = Counter(flatList)
                fitness += sum(count for count in counts.values() if count > 2)
                temp_list = []

        # Teacher Availability
        temp_list = []
        ctr = 0
        for chromosom in self.chromosome:
            temp_list.append(chromosom)
            ctr += 1
            if ctr%10 == 0 and ctr!=0:
                flatList = [element for innerList in temp_list for element in innerList] 
                for i in range(0,8):
                    teacher = []
                    for j in range(i,50,9):
                        teacher.append(df.loc[flatList[j]]['Teacher ID'])
                    counts = Counter(teacher)
                    fitness += sum(count for count in counts.values() if count > 1)

        # Section and semester Availability
        temp_list = []
        ctr = 0
        for chromosom in self.chromosome:
            temp_list.append(chromosom)
            ctr += 1
            if ctr%10 == 0 and ctr!=0:
                flatList = [element for innerList in temp_list for element in innerList] 
                for i in range(0,8):
                    sec_sem = []
                    for j in range(i,50,9):
                        sec_sem.append([df.loc[flatList[j]]['Section'],df.loc[flatList[j]]['Semester']])
                    counts = Counter(tuple(x) for x in sec_sem)
                    fitness += sum(count for count in counts.values() if count > 1)

        if fitness > 0:
            return fitness
        
        # Credit Hours Checking
        flatList = [element for innerList in self.chromosome for element in innerList]
        counts = Counter(flatList)
        fitness += sum(count for count in counts.values() if count > 3)

        # Wednesday free slot Checking
        wednesday = self.chromosome[18:29]
        wed_flatList = [element for innerList in wednesday for element in innerList]
        for i in range(4,50,9):
            if wed_flatList[i] == 102:
                fitness -= 3

        # Teacher and student Preferences
        pref = {4:[20, 21, 22, 23, 79,2,6,34,88,101],7:[[81,39,66,33,99]]}
        flatList = [element for innerList in self.chromosome for element in innerList]
        for i in range(0,50):
            if flatList[i] in pref[4]:
                if i%4 == 0 or i%3 == 0 or i%2 == 0:
                    fitness -= 1
            if flatList[i] in pref[7]:
                if i%7 == 0 or i%5 == 0 or i%6 == 0:
                    fitness -= 1
        
        print("Fitness: ", fitness)    
        return fitness
            
        

In [21]:
def main(): 
    global POPULATION_SIZE 
  
    #current generation 
    generation = 1
  
    found = False
    population = [] 
  
    # create initial population 
    for _ in range(POPULATION_SIZE): 
                gnome = Individual.create_gnome() 
                population.append(Individual(gnome)) 
  
    while not found: 
  
        # sort the population in increasing order of fitness score 
        population = sorted(population, key = lambda x:x.fitness) 
  
        # if the individual having lowest fitness score ie.  
        # 0 then we know that we have reached to the target 
        # and break the loop 
        if population[0].fitness <= 0: 
            found = True
            break
  
        # Otherwise generate new offsprings for new generation 
        new_generation = [] 
  
        # Perform Elitism, that mean 10% of fittest population 
        # goes to the next generation 
        s = int((30*POPULATION_SIZE)/100) 
        new_generation.extend(population[:s]) 
  
        # From 50% of fittest population, Individuals  
        # will mate to produce offspring 
        s = int((70*POPULATION_SIZE)/100) 
        for _ in range(s): 
            parent1 = random.choice(population[:50]) 
            parent2 = random.choice(population[:50]) 
            child = parent1.mate(parent2) 
            new_generation.append(child) 
  
        population = new_generation 
        print("Generation: {}\tString: {}\tFitness: {}",generation, population[0].fitness)
  
        generation += 1
  
      
    print("Generation: {}\tString: {}\tFitness: {}",generation, population[0].fitness)
    print(population[0].fitness)
    print(population[0].chromosome)
  
if __name__ == '__main__': 
    main() 

Generation: {}	String: {}	Fitness: {} 1 96
Generation: {}	String: {}	Fitness: {} 2 94
Generation: {}	String: {}	Fitness: {} 3 94
Generation: {}	String: {}	Fitness: {} 4 86
Generation: {}	String: {}	Fitness: {} 5 64
Generation: {}	String: {}	Fitness: {} 6 64
Generation: {}	String: {}	Fitness: {} 7 64
Generation: {}	String: {}	Fitness: {} 8 62
Generation: {}	String: {}	Fitness: {} 9 62
Generation: {}	String: {}	Fitness: {} 10 62
Generation: {}	String: {}	Fitness: {} 11 62
Generation: {}	String: {}	Fitness: {} 12 62
Generation: {}	String: {}	Fitness: {} 13 62
Generation: {}	String: {}	Fitness: {} 14 62
Generation: {}	String: {}	Fitness: {} 15 62
Generation: {}	String: {}	Fitness: {} 16 62
Generation: {}	String: {}	Fitness: {} 17 52
Generation: {}	String: {}	Fitness: {} 18 52
Generation: {}	String: {}	Fitness: {} 19 52
Generation: {}	String: {}	Fitness: {} 20 52
Generation: {}	String: {}	Fitness: {} 21 52
Generation: {}	String: {}	Fitness: {} 22 50
Generation: {}	String: {}	Fitness: {} 23 

TypeError: 'int' object is not iterable

In [ ]:
from collections import Counter

l = [['A', 2], ['B', 4],['A', 2]]
counts = Counter(tuple(x) for x in l)

print(counts)

Counter({('A', 2): 2, ('B', 4): 1})


In [22]:
for i in range(4,50,9):
    print(i)

4
13
22
31
40
49


In [28]:
gg = ['A','B','C']

k = gg[-1:2]

print(type(k))


<class 'list'>
